In [1]:
from pyspark.sql import SparkSession
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd

In [2]:
spark = SparkSession.builder.appName('vizualizações').getOrCreate()

23/11/17 20:52:30 WARN Utils: Your hostname, rodrigo-Nitro-AN515-54 resolves to a loopback address: 127.0.1.1; using 192.168.0.112 instead (on interface wlp8s0)
23/11/17 20:52:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/11/17 20:52:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/17 20:52:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark

In [4]:
df_parquet = spark.read.parquet('/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/assunto_cities_skylines/total_video_publicado_semana/total_video_publicado_semana.parquet')
df_parquet.createOrReplaceTempView('TOTAL_PUBLICACOES')

In [5]:
spark.sql('SELECT * FROM TOTAL_PUBLICACOES WHERE NM_CANAL LIKE "C%"').show(truncate=False)

+----------------+----------------------------+------------+---------------+------------------------+
|SEMANA_TRADUZIDA|NM_CANAL                    |TOTAL_VIDEOS|DATA_PUBLICACAO|ID_CANAL                |
+----------------+----------------------------+------------+---------------+------------------------+
|Segunda-feira   |CANAL HAND - GAME WITH MUSIC|1           |2023-10-16     |UC7v51W2NoLFawizgcNuVBOA|
|Segunda-feira   |CANAL HAND - GAME WITH MUSIC|1           |2023-10-23     |UC7v51W2NoLFawizgcNuVBOA|
|Quarta-feira    |CANAL HAND - GAME WITH MUSIC|1           |2023-10-18     |UC7v51W2NoLFawizgcNuVBOA|
|Terça-feira     |CANAL HAND - GAME WITH MUSIC|1           |2023-10-17     |UC7v51W2NoLFawizgcNuVBOA|
|Sexta-feira     |CANAL HAND - GAME WITH MUSIC|1           |2023-10-20     |UC7v51W2NoLFawizgcNuVBOA|
|Sábado          |CANAL HAND - GAME WITH MUSIC|1           |2023-10-21     |UC7v51W2NoLFawizgcNuVBOA|
|Domingo         |CANAL HAND - GAME WITH MUSIC|1           |2023-10-22     |UC7v51

In [6]:
spark.sql('SELECT DATA_PUBLICACAO, sum(TOTAL_VIDEOS) FROM TOTAL_PUBLICACOES GROUP BY  DATA_PUBLICACAO').show()

+---------------+-----------------+
|DATA_PUBLICACAO|sum(TOTAL_VIDEOS)|
+---------------+-----------------+
|     2023-10-25|               23|
|     2023-10-16|               16|
|     2023-10-15|               10|
|     2023-10-18|               14|
|     2023-10-23|               24|
|     2023-10-21|               21|
|     2023-10-20|               13|
|     2023-10-19|               17|
|     2023-10-22|               15|
|     2023-10-17|               16|
|     2023-10-26|               22|
|     2023-10-24|               23|
|     2023-10-27|               21|
+---------------+-----------------+



In [7]:
spark.sql('SELECT COUNT(*) FROM TOTAL_PUBLICACOES where DATA_PUBLICACAO = "2023-10-25"  ' ).show(26)

+--------+
|count(1)|
+--------+
|      23|
+--------+



# Relação Comentários X Visualização (proporção) e Relação Visualizações X Likes (proporção)

In [8]:
assunto = 'assunto_cities_skylines'

In [9]:
df_visualizacoes_semana = spark.read.parquet(f'/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/{assunto}/total_visualizacoes_por_semana')
df_visualizacoes_semana.createOrReplaceTempView('METRICAS_VIDEO')

In [10]:
spark.sql(' SELECT *'
          ' FROM METRICAS_VIDEO ').show(truncate=False)

+---------------+-----------+---------------------------------------------------------------------------------------------+---------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------+----------+-------------------+-----------------+-----------+-----------------------+---------------------+---------------+-------------+------------------------+
|NM_CANAL       |ID_VIDEO   |TITULO_VIDEO                                                                                 |TOTAL_CARACTERE_VIDEO|TAGS                

In [11]:
spark.sql(' SELECT ID_CANAL, NM_CANAL, count(*)'
          ' FROM METRICAS_VIDEO '
          ' GROUP BY ID_CANAL, NM_CANAL ORDER BY 3 DESC').show(truncate=False)

+------------------------+----------------------------+--------+
|ID_CANAL                |NM_CANAL                    |count(1)|
+------------------------+----------------------------+--------+
|UCrOH1V-FyMunBIMrKL0y0xQ|ChratosGameplay             |114     |
|UC7v51W2NoLFawizgcNuVBOA|CANAL HAND - GAME WITH MUSIC|83      |
|UCkXA2EoaXRFW18vrAE01BpQ|Betospa Gaming              |67      |
|UC1mk6EtfMjxR4eEZ7C43zTQ|Irmãos Cities               |67      |
|UCfY4ggUDPeecGSCNePyyVSA|MetalBear                   |58      |
|UCcn60fs5EMW8A_k9R-e4gEg|Cabrunco Games              |51      |
|UCOYJm7BsLV8VCkW2DC1MI5w|Lucas gameplays brasil      |51      |
|UCayiH2Y5JBEOSubqoQhufDA|Lives do Chratos            |48      |
|UCDGo_s0I-2JpO3o3xlkMQVA|Simularies                  |38      |
|UCrUcdH_bCfX77xpXHyq2fJw|Canal do Void               |37      |
|UCmg3_taw1BrGuo1ITlQe2hA|O Construtor                |36      |
|UCTrNnXivZU_1-aJqn1uNANg|DILSÃO                      |35      |
|UCe9jrI0YQ5SM6h5QRZ9FZlA

In [12]:
df_result = spark.sql(' SELECT ID_VIDEO, TOTAL_CARACTERE_VIDEO, TITULO_VIDEO,NM_CANAL, TOTAL_COMENTARIOS, TOTAL_LIKES , TOTAL_VISUALIZACOES '
          ' FROM METRICAS_VIDEO '
        #   ' WHERE data_extracao = "2023-10-27" ' 
          ' where ID_CANAL in ("UCrOH1V-FyMunBIMrKL0y0xQ") ').toPandas()
fig = px.scatter(df_result, x='TOTAL_CARACTERE_VIDEO', y='TOTAL_VISUALIZACOES')
df_result.sort_values(by='TOTAL_VISUALIZACOES', ascending=False).head()


,ID_VIDEO,TOTAL_CARACTERE_VIDEO,TITULO_VIDEO,NM_CANAL,TOTAL_COMENTARIOS,TOTAL_LIKES,TOTAL_VISUALIZACOES
13,wCLSZxLfUAk,75,MELHOR JOGO DE CONSTRUIR CIDADES CHEGOU E ESTÁ...,ChratosGameplay,418,13988,120812
31,wCLSZxLfUAk,75,MELHOR JOGO DE CONSTRUIR CIDADES CHEGOU E ESTÁ...,ChratosGameplay,416,13784,117375
47,wCLSZxLfUAk,75,MELHOR JOGO DE CONSTRUIR CIDADES CHEGOU E ESTÁ...,ChratosGameplay,414,13460,111842
62,wCLSZxLfUAk,75,MELHOR JOGO DE CONSTRUIR CIDADES CHEGOU E ESTÁ...,ChratosGameplay,408,13033,104409
74,wCLSZxLfUAk,75,MELHOR JOGO DE CONSTRUIR CIDADES CHEGOU E ESTÁ...,ChratosGameplay,403,12605,97039


In [13]:
fig.show()

- Comparação Likes X Visualizações e Cometários X Visualizações

In [14]:
fig = make_subplots(rows=1, cols=3, subplot_titles=['Likes X Visualizações', 'Cometários X Visualizações', 'TOTAL_CARACTERE_VIDEO X VISUALIZAÇÔES'])
sc1 = px.scatter(df_result, x='TOTAL_LIKES', y='TOTAL_VISUALIZACOES',)
fig.add_trace(sc1['data'][0], row=1, col=1)
fig.update_xaxes(title_text='Likes', row=1, col=1)
fig.update_yaxes(title_text='Visualizações', row=1, col=1)

sc2 = px.scatter(df_result, x='TOTAL_COMENTARIOS', y='TOTAL_VISUALIZACOES')
fig.add_trace(sc2['data'][0], row=1, col=2)
fig.update_xaxes(title_text='Comentários', row=1, col=2)
fig.update_yaxes(title_text='Visualizações', row=1, col=2)
fig.update_layout(title_text='Desenpenho Canal', showlegend=True)

sc3 = px.scatter(df_result, x='TOTAL_CARACTERE_VIDEO', y='TOTAL_VISUALIZACOES')
fig.add_trace(sc3['data'][0], row=1, col=3)
fig.update_xaxes(title_text='TOTAL_CARACTERE_VIDEO', row=1, col=3)
fig.update_yaxes(title_text='Visualizações', row=1, col=3)
fig.update_layout(title_text='Desenpenho Canal', showlegend=True)



fig.show()

# Dia em que os vídeos são Públicados com alta frequência

In [15]:
df_dias_publicado = spark.read.parquet(f'/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/{assunto}/total_video_publicado_semana')
df_dias_publicado.createOrReplaceTempView('PUBLICACAO_VIDEO')

In [16]:
spark.sql('select * from PUBLICACAO_VIDEO WHERE ID_CANAL IN ("UCrOH1V-FyMunBIMrKL0y0xQ")').show()

+----------------+---------------+------------+---------------+--------------------+
|SEMANA_TRADUZIDA|       NM_CANAL|TOTAL_VIDEOS|DATA_PUBLICACAO|            ID_CANAL|
+----------------+---------------+------------+---------------+--------------------+
|   Segunda-feira|ChratosGameplay|           1|     2023-10-23|UCrOH1V-FyMunBIMr...|
|   Segunda-feira|ChratosGameplay|           1|     2023-10-16|UCrOH1V-FyMunBIMr...|
|     Terça-feira|ChratosGameplay|           1|     2023-10-24|UCrOH1V-FyMunBIMr...|
|    Quinta-feira|ChratosGameplay|           1|     2023-10-26|UCrOH1V-FyMunBIMr...|
|    Quinta-feira|ChratosGameplay|           1|     2023-10-19|UCrOH1V-FyMunBIMr...|
|    Quarta-feira|ChratosGameplay|           1|     2023-10-18|UCrOH1V-FyMunBIMr...|
|     Terça-feira|ChratosGameplay|           1|     2023-10-17|UCrOH1V-FyMunBIMr...|
|    Quarta-feira|ChratosGameplay|           1|     2023-10-25|UCrOH1V-FyMunBIMr...|
|     Sexta-feira|ChratosGameplay|           1|     2023-10-20|UC

In [17]:
def indice_semana(dia: str) -> int: 
    dias_semana = {
        'Domingo': 1,
        'Segunda-feira' : 2,
        'Terça-feira' : 3,
        'Quarta-feira' : 4,
        'Quinta-feira': 5,
        'Sexta-feira': 6,
        'Sábado' : 7
    }

    return dias_semana.get(dia)


In [18]:
indice_semana('Sábado')

7

In [19]:
df_publicacao_video  = spark.sql(' SELECT SEMANA_TRADUZIDA,  sum(TOTAL_VIDEOS) '
          ' FROM PUBLICACAO_VIDEO'
          ' GROUP BY SEMANA_TRADUZIDA ').toPandas()
df_publicacao_video['INDICE_SEMANA'] = df_publicacao_video['SEMANA_TRADUZIDA'].apply(indice_semana)
df_publicacao_video.sort_values(by='INDICE_SEMANA', inplace=True)
df_publicacao_video.head(7)


,SEMANA_TRADUZIDA,sum(TOTAL_VIDEOS),INDICE_SEMANA
5,Domingo,25,1
0,Segunda-feira,40,2
2,Terça-feira,39,3
1,Quarta-feira,37,4
6,Quinta-feira,39,5
4,Sexta-feira,34,6
3,Sábado,21,7


In [20]:
fig = px.bar(df_publicacao_video, x='SEMANA_TRADUZIDA',y='sum(TOTAL_VIDEOS)', text_auto=True)
fig.update_layout(title_text='Envio de Vídeo por Semana', showlegend=True, title=dict(x=0.5) )

fig.show()

# Total de Tags X Vísualizações 

In [21]:
df_tags_visualizacoes = spark.sql('SELECT TOTAL_TAGS, TOTAL_VISUALIZACOES FROM METRICAS_VIDEO where ID_CANAL in ("UCrOH1V-FyMunBIMrKL0y0xQ") ').toPandas()
df_tags_visualizacoes.head()
fig = px.scatter(df_tags_visualizacoes, x='TOTAL_TAGS', y='TOTAL_VISUALIZACOES')
fig.show()

# Tamanho dos títulos do Vídeos X Vísualizações

# Total Víews DIA por Vídeo

In [38]:
df_views_video = spark.sql(' SELECT TITULO_VIDEO, TOTAL_VISUALIZACOES_DIA '  
          ' FROM METRICAS_VIDEO ' 
          ' WHERE ID_CANAL = "UCrOH1V-FyMunBIMrKL0y0xQ" ' 
          ' AND data_extracao = "2023-10-26" '
          ' AND TOTAL_VISUALIZACOES > 0 '
          ' ORDER BY 2 ').toPandas()
df_views_video.head() 
df_views_video['TITULO_VIDEO'] = df_views_video['TITULO_VIDEO'].apply(lambda x: x.capitalize())
fig = px.bar(df_views_video, x='TOTAL_VISUALIZACOES_DIA', y='TITULO_VIDEO', orientation='h', text_auto=True)
fig.show()


# Total Víews DIA por Canal

In [23]:
df_views_canal = spark.sql(
    ' SELECT NM_CANAL, sum(TOTAL_VISUALIZACOES_DIA)  '
    ' FROM  METRICAS_VIDEO '
    ' WHERE  data_extracao = "2023-10-27"  AND TOTAL_VISUALIZACOES_DIA > 0 '
    ' GROUP BY NM_CANAL '
    ' ORDER BY 2 '
).toPandas()
df_views_canal['NM_CANAL'] = df_views_canal['NM_CANAL'].apply(lambda x: x.capitalize())

df_views_canal.head()
fig = px.bar(df_views_canal, x='sum(TOTAL_VISUALIZACOES_DIA)', y='NM_CANAL', orientation='h', text_auto=True)
fig.show() 

# A duração do vídeo é importante para visualizações e interação (curtidas/comentários)?

In [24]:
df_duracao_video = spark.sql(
    ' SELECT DURACAO_VIDEO_MINUTOS, TOTAL_COMENTARIOS, TOTAL_LIKES, TOTAL_VISUALIZACOES'
    ' FROM METRICAS_VIDEO '
    ' WHERE ID_CANAL IN ("UCrOH1V-FyMunBIMrKL0y0xQ") '
    ).toPandas()
df_duracao_video.head()

,DURACAO_VIDEO_MINUTOS,TOTAL_COMENTARIOS,TOTAL_LIKES,TOTAL_VISUALIZACOES
0,0.000000,0,50,0
1,26.879999,220,5600,42650
2,25.670000,213,6855,54375
3,28.430000,124,6448,50789
4,32.279999,180,5046,39265


In [25]:
fig = make_subplots(rows=1, cols=3, subplot_titles=['DURACAO_VIDEO_MINUTOS X TOTAL_LIKES', 'DURACAO_VIDEO_MINUTOS X TOTAL_COMENTARIOS', 'DURACAO_VIDEO_MINUTOS X TOTAL_VISUALIZACOES'])
sc1 = px.scatter(df_duracao_video, x='DURACAO_VIDEO_MINUTOS', y='TOTAL_LIKES',)
fig.add_trace(sc1['data'][0], row=1, col=1)
fig.update_xaxes(title_text='DURACAO_VIDEO_MINUTOS', row=1, col=1)
fig.update_yaxes(title_text='TOTAL_LIKES', row=1, col=1)

sc2 = px.scatter(df_duracao_video, x='DURACAO_VIDEO_MINUTOS', y='TOTAL_COMENTARIOS')
fig.add_trace(sc2['data'][0], row=1, col=2)
fig.update_xaxes(title_text='DURACAO_VIDEO_MINUTOS', row=1, col=2)
fig.update_yaxes(title_text='TOTAL_COMENTARIOS', row=1, col=2)
fig.update_layout(title_text='Desenpenho Canal', showlegend=True)

sc3 = px.scatter(df_duracao_video, x='DURACAO_VIDEO_MINUTOS', y='TOTAL_VISUALIZACOES')
fig.add_trace(sc3['data'][0], row=1, col=3)
fig.update_xaxes(title_text='DURACAO_VIDEO_MINUTOS', row=1, col=3)
fig.update_yaxes(title_text='TOTAL_VISUALIZACOES', row=1, col=3)
fig.update_layout(title_text='Desenpenho Canal', showlegend=True)


# Desepenho dos vídeos (like, comentários, vísualizações) (Faça comparação)

In [26]:
df_duracao_video = spark.sql(
    ' SELECT data_extracao, NM_CANAL, SUM(TOTAL_VISUALIZACOES_DIA) '
    ' FROM METRICAS_VIDEO '
    ' WHERE  ID_VIDEO IN ("xCaUzgIM6Bg")  '
    ' GROUP BY data_extracao, NM_CANAL '
    ).show()

+-------------+---------------+----------------------------+
|data_extracao|       NM_CANAL|sum(TOTAL_VISUALIZACOES_DIA)|
+-------------+---------------+----------------------------+
|   2023-10-27|ChratosGameplay|                        2388|
|   2023-10-25|ChratosGameplay|                        4298|
|   2023-10-26|ChratosGameplay|                       11934|
+-------------+---------------+----------------------------+



In [27]:
df_duracao_video = spark.sql(
    ' SELECT data_extracao, ID_CANAL, TITULO_VIDEO, TOTAL_VISUALIZACOES_DIA, TOTAL_LIKES_DIA, TOTAL_COMENTARIOS_DIA '
    ' FROM METRICAS_VIDEO '
    ' WHERE  ID_VIDEO IN ("xCaUzgIM6Bg")  '
    ' ORDER BY 1 '
    ).show(truncate=False)

+-------------+------------------------+--------------------------------------------------------------------------------------+-----------------------+---------------+---------------------+
|data_extracao|ID_CANAL                |TITULO_VIDEO                                                                          |TOTAL_VISUALIZACOES_DIA|TOTAL_LIKES_DIA|TOTAL_COMENTARIOS_DIA|
+-------------+------------------------+--------------------------------------------------------------------------------------+-----------------------+---------------+---------------------+
|2023-10-25   |UCrOH1V-FyMunBIMrKL0y0xQ|Vias expressas/Rodovias e aneis rodoviários - Guia de planejamento - Cities Skylines 2|4298                   |633            |30                   |
|2023-10-26   |UCrOH1V-FyMunBIMrKL0y0xQ|Vias expressas/Rodovias e aneis rodoviários - Guia de planejamento - Cities Skylines 2|11934                  |635            |21                   |
|2023-10-27   |UCrOH1V-FyMunBIMrKL0y0xQ|Vias expre

# Desepenho dos Canais (like, comentários, vísualizações) Faça Comparação


In [28]:
df_duracao_video = spark.sql(
    ' SELECT data_extracao, NM_CANAL, SUM(TOTAL_VISUALIZACOES_DIA) as SOMA_VISUALIZACOES_DIA, sum(TOTAL_LIKES_DIA) AS SOMA_TOTAL_LIKES_DIA, sum(TOTAL_COMENTARIOS_DIA) AS SOMA_TOTAL_COMENTARIOS_DIA'
    ' FROM METRICAS_VIDEO '
    ' WHERE  ID_CANAL IN ("UCrOH1V-FyMunBIMrKL0y0xQ" , "UCe9jrI0YQ5SM6h5QRZ9FZlA")  '
    ' GROUP BY data_extracao, NM_CANAL '
    ' ORDER BY 1 '
    ).toPandas()

df_duracao_video.head()

,data_extracao,NM_CANAL,SOMA_VISUALIZACOES_DIA,SOMA_TOTAL_LIKES_DIA,SOMA_TOTAL_COMENTARIOS_DIA
0,2023-10-15,ChratosGameplay,4562,687,23
1,2023-10-16,ChratosGameplay,10751,1753,61
2,2023-10-19,ChratosGameplay,66844,10675,343
3,2023-10-19,Barbosa Cities,12830,2806,168
4,2023-10-20,ChratosGameplay,50950,7585,421


In [29]:
fig = px.bar(df_duracao_video, x='data_extracao', y='SOMA_VISUALIZACOES_DIA', barmode='group', color='NM_CANAL')
fig.show()



In [30]:
fig = px.bar(df_duracao_video, x='data_extracao', y='SOMA_TOTAL_LIKES_DIA', barmode='group', color='NM_CANAL')
fig.show()

In [31]:
fig = px.bar(df_duracao_video, x='data_extracao', y='SOMA_TOTAL_COMENTARIOS_DIA', barmode='group', color='NM_CANAL')
fig.show()

# Wordcloud TAGS

# Métricas especificas TRENDS

In [32]:
df_trends = spark.read.parquet('/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/trends_brazil/trends_brazil/trends_brazil.parquet')
df_trends.show(truncate=False)
df_trends.createOrReplaceTempView('TRENDS')

+------------------------------------------------+------------------------------------------------------------------------------------------------------+-------------------+---------------+-----------------+-----------+-----------------------+-------------------+---------------------+---------------+-------------+------------+------------------------+-----------+
|NM_CANAL                                        |TITULO_VIDEO                                                                                          |TOTAL_VISUALIZACOES|TOTAL_FAVORITOS|TOTAL_COMENTARIOS|TOTAL_LIKES|TOTAL_VISUALIZACOES_DIA|TOTAL_FAVORITOS_DIA|TOTAL_COMENTARIOS_DIA|TOTAL_LIKES_DIA|data_extracao|ID_CATEGORIA|ID_CANAL                |ID_VIDEO   |
+------------------------------------------------+------------------------------------------------------------------------------------------------------+-------------------+---------------+-----------------+-----------+-----------------------+-------------------+-----

# Categrias Populares por data de extração

In [33]:
df_categoria_populares = spark.sql(' SELECT data_extracao, ID_CATEGORIA, SUM(TOTAL_VISUALIZACOES_DIA) '
                                  '  FROM TRENDS '
                                  '  GROUP BY data_extracao, ID_CATEGORIA  '
                                  '  ORDER BY 1 '   ).show()

+-------------+------------+----------------------------+
|data_extracao|ID_CATEGORIA|sum(TOTAL_VISUALIZACOES_DIA)|
+-------------+------------+----------------------------+
|   2023-10-15|          17|                    25971018|
|   2023-10-15|          10|                   188470120|
|   2023-10-15|          26|                       49953|
|   2023-10-15|          25|                    11520764|
|   2023-10-15|           1|                    10646952|
|   2023-10-15|          27|                     5215689|
|   2023-10-15|          22|                     6772581|
|   2023-10-15|          20|                    29314189|
|   2023-10-15|          28|                     6273526|
|   2023-10-15|          23|                     1361139|
|   2023-10-15|          24|                   285267745|
|   2023-10-16|          20|                     8515606|
|   2023-10-16|          27|                      256909|
|   2023-10-16|           1|                      981247|
|   2023-10-16

In [34]:
df_categoria_populares = spark.sql(' SELECT data_extracao, ID_CATEGORIA, SUM(TOTAL_COMENTARIOS_DIA) '
                                  '  FROM TRENDS '
                                  '  GROUP BY data_extracao, ID_CATEGORIA  '
                                  '  ORDER BY 1 '   ).show()

+-------------+------------+--------------------------+
|data_extracao|ID_CATEGORIA|sum(TOTAL_COMENTARIOS_DIA)|
+-------------+------------+--------------------------+
|   2023-10-15|          17|                     50166|
|   2023-10-15|          10|                    505272|
|   2023-10-15|          26|                       690|
|   2023-10-15|          25|                     13235|
|   2023-10-15|           1|                     36792|
|   2023-10-15|          27|                     12350|
|   2023-10-15|          22|                     49841|
|   2023-10-15|          20|                    116548|
|   2023-10-15|          28|                     14408|
|   2023-10-15|          23|                      1486|
|   2023-10-15|          24|                    572240|
|   2023-10-16|          20|                     28578|
|   2023-10-16|          27|                      2881|
|   2023-10-16|           1|                      1812|
|   2023-10-16|          24|                    

In [35]:
df_categoria_populares = spark.sql(' SELECT data_extracao, ID_CATEGORIA, SUM(TOTAL_LIKES_DIA) '
                                  '  FROM TRENDS '
                                  '  GROUP BY data_extracao, ID_CATEGORIA  '
                                  '  ORDER BY 1 '   ).show()

+-------------+------------+--------------------+
|data_extracao|ID_CATEGORIA|sum(TOTAL_LIKES_DIA)|
+-------------+------------+--------------------+
|   2023-10-15|          17|              942816|
|   2023-10-15|          10|             8054410|
|   2023-10-15|          26|                9076|
|   2023-10-15|          25|              248596|
|   2023-10-15|           1|              487262|
|   2023-10-15|          27|              263761|
|   2023-10-15|          22|              620345|
|   2023-10-15|          20|             1747021|
|   2023-10-15|          28|                2956|
|   2023-10-15|          23|               68949|
|   2023-10-15|          24|            11743071|
|   2023-10-16|          20|              487937|
|   2023-10-16|          27|               19982|
|   2023-10-16|           1|               38297|
|   2023-10-16|          24|              871289|
|   2023-10-16|          22|               91885|
|   2023-10-16|           2|               19070|


# Vídeo populares

In [36]:
df_video_populares = spark.sql(' SELECT data_extracao, TITULO_VIDEO, SUM(TOTAL_VISUALIZACOES_DIA) '
                                  '  FROM TRENDS '
                                  '  GROUP BY data_extracao, TITULO_VIDEO  '
                                  '  ORDER BY 1 '   ).show()

+-------------+--------------------+----------------------------+
|data_extracao|        TITULO_VIDEO|sum(TOTAL_VISUALIZACOES_DIA)|
+-------------+--------------------+----------------------------+
|   2023-10-15|🔴 FLAMENGO ACERT...|                      106760|
|   2023-10-15|FÁCIL COMO NENHUM...|                      525570|
|   2023-10-15|PSG É HUMILHADO P...|                     3106648|
|   2023-10-15|😱 ABRIU O ARMÁRI...|                      386196|
|   2023-10-15|Goblin Mines Dist...|                     2002737|
|   2023-10-15|A BRUX4 TÁ SOLTA!...|                       80215|
|   2023-10-15|PALMEIRAS 6 X 0 O...|                      127723|
|   2023-10-15|CHEGOU! CONHECEND...|                       98244|
|   2023-10-15|EPISÓDIO 04 - 3/5...|                     1422581|
|   2023-10-15|A MORTE de G-MAN!...|                      113073|
|   2023-10-15|O FAMOSO BOLO ALA...|                     1756861|
|   2023-10-15|MEU DEUS! "OLHA I...|                      390493|
|   2023-10-

# Desenpenho Vídeo

In [37]:
df_video_populares = spark.sql(' SELECT data_extracao, TITULO_VIDEO, TOTAL_VISUALIZACOES_DIA'
                                  '  FROM TRENDS '
                                  ' WHERE ID_VIDEO IN ("j4TlIePiewE") '
                                  '  ORDER BY 1 '   ).show()

+-------------+--------------------+-----------------------+
|data_extracao|        TITULO_VIDEO|TOTAL_VISUALIZACOES_DIA|
+-------------+--------------------+-----------------------+
|   2023-10-25|REAL SOCIEDAD VEN...|                 121071|
|   2023-10-26|REAL SOCIEDAD VEN...|                   3115|
|   2023-10-27|REAL SOCIEDAD VEN...|                   1218|
+-------------+--------------------+-----------------------+

